In [3]:
# Import Libraries
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import time


In [4]:
def getDriver():
    # Set up Selenium WebDriver with the ChromeDriver executable path
    driver_path = r'C:/Users/hari_/Downloads/chromedriver_win32/chromedriver.exe'

    # Set the download path
    download_path = r'D:/github/1-Projects/Research/SSBM/IndianKannon/PDF_Judgments_Ecourts'

    # Configure Chrome options
    chrome_options = Options()
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--headless')  # Run Chrome in headless mode
    chrome_options.add_argument(f'--download.default_directory={download_path}')

    pref = {
        'download.default_directory': download_path,
        'download.prompt_for_download': False,
        'download.directory_upgrade': True,
        'safebrowsing.enabled': True,
        "plugins.always_open_pdf_externally": True
    }

    chrome_options.add_experimental_option('prefs', pref)

    # Initialize the driver with the configured options
    driver = webdriver.Chrome(service=Service(driver_path), options=chrome_options)
    return driver 

In [5]:
# This code doesnot work because this website uses number captcha.

# Specify the URL of the website
url = "https://judgments.ecourts.gov.in/"

# Specify the act and section you want to search
act = "INDIAN PENAL CODE"
section = "375"

driver = getDriver()

# Open the website
driver.get(url)

# Find and fill in the act and section search fields
def find_act_section(driver):
    driver.find_element(By.ID, "act_txt").send_keys(act)
    driver.find_element(By.ID, "section_txt").send_keys(section)

    # # Submit the form
    # driver.find_element(By.ID, "actsecSearch").submit()

    # Click on the search button using JavaScript
    search_button = driver.find_element(By.CSS_SELECTOR, ".fa-search").find_element(By.XPATH, "..")
    driver.execute_script("arguments[0].click();", search_button)


    # Wait for the search results to load
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".jud_cname")))

    # Loop through the pages and download the PDF files
    page_number = 1
    while True:
        # Find all the links on the page
        links = driver.find_elements(By.CSS_SELECTOR, ".jud_cname a")
        
        # Download the PDF files from the links
        for link in links:
            link.click()
            time.sleep(1)  # Wait for the file to download
        
        # Check if there is a next page
        next_button = driver.find_element(By.XPATH, "//a[contains(text(), 'Next')]")
        if "disabled" in next_button.get_attribute("class"):
            break
        
        # Go to the next page
        page_number += 1
        driver.get(f"{url}?act={act}&section={section}&page={page_number}")

# find_act_section(driver)

# Close the webdriver
driver.quit()

In [ ]:
import requests
# Specify the URL of the website
url = "https://judgments.ecourts.gov.in/"

# Specify your 2Captcha API key
# 2Captcha (https://2captcha.com/)
api_key = "your_2captcha_api_key"

# Open the website with the CAPTCHA
driver = getDriver()

# Open the website
driver.get(url)

# Wait for the CAPTCHA to load
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "captcha")))

# Solve the CAPTCHA using 2Captcha API
captcha_image_url = driver.find_element(By.ID, "captcha").get_attribute("src")
response = requests.get(captcha_image_url, stream=True)
captcha_image_path = "/path/to/save/captcha_image.jpg"  # Specify the path to save the CAPTCHA image
with open(captcha_image_path, "wb") as file:
    file.write(response.content)

# Send the CAPTCHA image to 2Captcha for solving
captcha_api_url = "http://2captcha.com/in.php"
data = {
    "key": api_key,
    "method": "post",
    "file": captcha_image_path,
}
response = requests.post(captcha_api_url, data=data)
captcha_id = response.text.split("|")[1]

# Get the solved CAPTCHA from 2Captcha
captcha_solution_url = f"http://2captcha.com/res.php?key={api_key}&action=get&id={captcha_id}"
while True:
    response = requests.get(captcha_solution_url)
    if response.text.startswith("CAPCHA_NOT_READY"):
        time.sleep(5)
    else:
        captcha_solution = response.text.split("|")[1]
        break

# Enter the CAPTCHA solution in the input field
captcha_input = driver.find_element(By.ID, "captcha")
captcha_input.send_keys(captcha_solution)

# Click on the submit button to proceed
submit_button = driver.find_element(By.XPATH, "//button[contains(text(), 'Submit')]")
submit_button.click()

# Wait for the search results to load
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".jud_cname")))


# This code doesnot work because this website uses number captcha.

# Specify the act and section you want to search
act = "INDIAN PENAL CODE"
section = "375"

find_act_section(driver)

# Close the webdriver
driver.quit()

